# 2️⃣ Re-Imaging Price Trends - 모델 학습 & 평가

**전제조건**: `1_image_generation.ipynb` 완료

**실행 내용**: CNN 모델 학습 → 포트폴리오 성과 평가 → 결과 시각화

In [ ]:
# 환경 설정
!pip install -r requirements.txt

from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/ReImaging_Price_Trends')
print(f"📁 현재 디렉토리: {os.getcwd()}")

# GPU 확인
import torch
print(f"🖥️ GPU: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'CPU'}")

# GPU 메모리 모니터링 및 관리 함수 정의
def check_gpu_memory():
    """GPU 메모리 상태 확인 및 출력"""
    if torch.cuda.is_available():
        device = torch.cuda.current_device()
        total_memory = torch.cuda.get_device_properties(device).total_memory
        allocated = torch.cuda.memory_allocated(device)
        cached = torch.cuda.memory_reserved(device)
        
        print(f"🖥️ GPU 메모리 상태:")
        print(f"   전체: {total_memory/1024**3:.1f}GB")
        print(f"   사용중: {allocated/1024**3:.1f}GB ({allocated/total_memory*100:.1f}%)")
        print(f"   캐시: {cached/1024**3:.1f}GB ({cached/total_memory*100:.1f}%)")
        print(f"   여유: {(total_memory-allocated)/1024**3:.1f}GB")
        
        return allocated, total_memory
    else:
        print("🖥️ GPU 사용 불가 - CPU 모드")
        return 0, 0

def cleanup_memory():
    """메모리 정리 및 가비지 컬렉션"""
    import gc
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        torch.cuda.synchronize()
    gc.collect()
    print("🧹 메모리 정리 완료")

# 초기 GPU 메모리 상태 확인
print("🔍 초기 GPU 메모리 상태:")
check_gpu_memory()
cleanup_memory()

In [ ]:
# 원본 형식 이미지 확인 (img_data_reconstructed)
import os

# Colab 환경에서의 절대경로 사용
base_path = '/content/drive/MyDrive/ReImaging_Price_Trends' if 'google.colab' in str(get_ipython()) else '.'
original_data_dir = os.path.join(base_path, 'img_data_reconstructed')

print(f"📁 원본 데이터 경로: {original_data_dir}")

# 원본 형식(.dat + .feather) 디렉토리 확인
required_dirs = {
    '5d (weekly)': 'weekly_5d',
    '20d (monthly)': 'monthly_20d', 
    '60d (quarterly)': 'quarterly_60d'
}

all_ready = True
for desc, dir_name in required_dirs.items():
    dir_path = os.path.join(original_data_dir, dir_name)
    if os.path.exists(dir_path):
        # 몇 개 연도 파일 확인
        sample_files = [f for f in os.listdir(dir_path) if f.endswith('.dat')][:3]
        print(f"✅ {desc}: {len(sample_files)}개 연도 파일 예시")
        for f in sample_files:
            print(f"   - {f}")
    else:
        print(f"❌ {desc}: {dir_path}")
        all_ready = False

if all_ready:
    print(f"\n🎉 원본 형식 데이터 준비 완료!")
    print("   논문 저자와 동일한 .dat + .feather 형식 사용")
    print("   --use_original_format 플래그로 실행됩니다")
else:
    print(f"\n⚠️ 일부 원본 데이터가 없습니다")
    print("create_original_format.py로 생성하거나 HDF5 형식 사용하세요")

## 🧠 CNN 모델 학습 (논문 방식 앙상블 지원)

**사용 형식**: `.dat` + `.feather` (논문 저자와 동일)  
**데이터 위치**: `img_data_reconstructed/`  
**앙상블**: 논문에서 언급한 5모델 평균 지원  

### 학습 방식 선택:
1. **단일 모델**: 빠른 테스트용
2. **앙상블 모델**: 논문과 동일한 5모델 평균 (더 안정적)

In [ ]:
# CNN5d 모델 학습 선택

print("🧠 CNN5d 학습 방식 선택:")
print("   단일 모델: 빠름 (1개 모델)")
print("   앙상블: 논문 방식 (5개 모델 평균)")

# 🔧 학습 방식 설정 (변경 가능)
use_ensemble = False  # True로 변경하면 앙상블 학습

print("GPU 메모리 상태 (학습 전):")
check_gpu_memory()

if use_ensemble:
    print("\n🔥 CNN5d 앙상블 학습 (5개 모델)...")
    print("⚠️ 시간이 5배 더 걸립니다!")
    !python ensemble_train.py --model CNN5d --image_days 5 --pred_days 5 --ensemble_runs 5 --use_original_format
else:
    print("\n🧠 CNN5d 단일 모델 학습...")
    !python main.py --model CNN5d --image_days 5 --pred_days 5 --use_original_format

print("\nGPU 메모리 상태 (학습 후):")
check_gpu_memory()
cleanup_memory()
print("✅ CNN5d 학습 완료")

In [ ]:
# CNN20d 모델 학습 선택

print("🧠 CNN20d 학습 방식 선택:")
print("   단일 모델: 빠름 (1개 모델)")
print("   앙상블: 논문 방식 (5개 모델 평균)")

# 🔧 학습 방식 설정 (변경 가능)
use_ensemble = False  # True로 변경하면 앙상블 학습

print("GPU 메모리 상태 (학습 전):")
check_gpu_memory()

if use_ensemble:
    print("\n🔥 CNN20d 앙상블 학습 (5개 모델)...")
    print("⚠️ 시간이 5배 더 걸립니다!")
    !python ensemble_train.py --model CNN20d --image_days 20 --pred_days 20 --ensemble_runs 5 --use_original_format
else:
    print("\n🧠 CNN20d 단일 모델 학습...")
    !python main.py --model CNN20d --image_days 20 --pred_days 20 --use_original_format

print("\nGPU 메모리 상태 (학습 후):")
check_gpu_memory()
cleanup_memory()
print("✅ CNN20d 학습 완료")

In [ ]:
# CNN60d 모델 학습 선택 (메모리 집약적)

print("🧠 CNN60d 학습 방식 선택 (⚠️ 메모리 집약적):")
print("   단일 모델: 빠름 (1개 모델)")
print("   앙상블: 논문 방식 (5개 모델 평균)")

# 🔧 학습 방식 설정 (변경 가능)
use_ensemble = False  # True로 변경하면 앙상블 학습

print("GPU 메모리 상태 (학습 전):")
allocated, total = check_gpu_memory()

# 메모리 부족 경고
if allocated > total * 0.7:  # 70% 이상 사용 중이면 경고
    print("⚠️ GPU 메모리 부족 위험! 추가 정리 수행...")
    cleanup_memory()

if use_ensemble:
    print("\n🔥 CNN60d 앙상블 학습 (5개 모델)...")
    print("⚠️ 시간이 5배 더 걸립니다!")
    !python ensemble_train.py --model CNN60d --image_days 60 --pred_days 60 --ensemble_runs 5 --use_original_format --batch_size 64
else:
    print("\n🧠 CNN60d 단일 모델 학습...")
    !python main.py --model CNN60d --image_days 60 --pred_days 60 --use_original_format --batch_size 64

print("\nGPU 메모리 상태 (학습 후):")
check_gpu_memory()
cleanup_memory()
print("✅ CNN60d 학습 완료")

# 모든 모델 평가 (원본 형식)
print("📈 포트폴리오 성과 평가 시작 (원본 .dat + .feather 형식)...\n")

print("1️⃣ CNN5d (Weekly Strategy)")
!python test.py --model CNN5d --image_days 5 --pred_days 5 --use_original_format
cleanup_memory()

print("\n2️⃣ CNN20d (Monthly Strategy)")
!python test.py --model CNN20d --image_days 20 --pred_days 20 --use_original_format
cleanup_memory()

print("\n3️⃣ CNN60d (Quarterly Strategy)")
!python test.py --model CNN60d --image_days 60 --pred_days 60 --use_original_format
cleanup_memory()

print("\n📊 모든 평가 완료!")
check_gpu_memory()

In [ ]:
# 📈 포트폴리오 성과 평가 (앙상블 지원)

print("📈 포트폴리오 성과 평가 시작...\n")
print("🔧 평가 방식:")
print("   단일 모델: 일반 예측")
print("   앙상블: 5모델 평균 예측 (더 안정적)")

# 🔧 평가 방식 설정 (학습 방식과 일치시키세요)
use_ensemble_eval = False  # 앙상블로 학습했다면 True로 변경

print("1️⃣ CNN5d (Weekly Strategy)")
if use_ensemble_eval:
    !python ensemble_test.py --model CNN5d --image_days 5 --pred_days 5 --use_original_format
else:
    !python test.py --model CNN5d --image_days 5 --pred_days 5 --use_original_format
cleanup_memory()

print("\n2️⃣ CNN20d (Monthly Strategy)")
if use_ensemble_eval:
    !python ensemble_test.py --model CNN20d --image_days 20 --pred_days 20 --use_original_format
else:
    !python test.py --model CNN20d --image_days 20 --pred_days 20 --use_original_format
cleanup_memory()

print("\n3️⃣ CNN60d (Quarterly Strategy)")
if use_ensemble_eval:
    !python ensemble_test.py --model CNN60d --image_days 60 --pred_days 60 --use_original_format
else:
    !python test.py --model CNN60d --image_days 60 --pred_days 60 --use_original_format
cleanup_memory()

print("\n📊 모든 평가 완료!")
check_gpu_memory()

## 결과 시각화

In [ ]:
# 최종 결과 요약 & GPU 메모리 분석
print("🎯 Re-Imaging Price Trends - 실행 완료!")
print("=" * 50)

# GPU 메모리 분석 결과
print("\n🖥️ GPU 메모리 요구사항 (배치크기 128):")
print("   • CNN5d:  ~0.02GB (가장 가볍다)")
print("   • CNN20d: ~0.09GB (중간)")  
print("   • CNN60d: ~0.34GB (가장 무겁다)")
print("\n💡 실제 사용량은 데이터 로딩 + optimizer state로 2-3배 더 클 수 있음")

print("\n📱 Colab GPU 호환성:")
print("   • T4 (16GB): 모든 모델 학습 가능")
print("   • V100/A100: 대용량 배치 처리 가능")
print("   • CNN60d는 배치크기 64 권장")

# 파일 크기 요약
if os.path.exists('models'):
    print("\n🧠 학습된 모델:")
    for file in sorted(os.listdir('models')):
        if file.endswith('.tar'):
            size_mb = os.path.getsize(f'models/{file}') / (1024**2)
            print(f"   ✅ {file} ({size_mb:.1f}MB)")

# 논문 성과와 비교
print("\n📚 논문 예상 성과 (비교 참조):")
print("   • Weekly (I5R5): H-L Sharpe = 7.15")
print("   • Monthly (I20R20): H-L Sharpe = 2.16") 
print("   • Quarterly (I60R60): H-L Sharpe = 0.37")

print("\n🔧 사용된 형식:")
print("   • 원본 형식 (.dat + .feather): 논문 저자와 동일")
print("   • 메모리 효율적이고 Colab 친화적")

print("\n✅ 위의 포트폴리오 평가 결과와 비교해보세요!")
print("\n💾 최종 GPU 메모리 상태:")
check_gpu_memory()

In [ ]:
# 최종 결과 요약 & 앙상블 정보

print("🎯 Re-Imaging Price Trends - 실행 완료!")
print("=" * 60)

# GPU 메모리 분석 결과
print("\n🖥️ GPU 메모리 요구사항 (배치크기 128):")
print("   • CNN5d:  ~0.02GB (가장 가볍다)")
print("   • CNN20d: ~0.09GB (중간)")  
print("   • CNN60d: ~0.34GB (가장 무겁다)")
print("\n💡 실제 사용량은 데이터 로딩 + optimizer state로 2-3배 더 클 수 있음")

print("\n📱 Colab GPU 호환성:")
print("   • T4 (16GB): 모든 모델 학습 가능")
print("   • V100/A100: 대용량 배치 처리 가능")
print("   • CNN60d는 배치크기 64 권장")

# 앙상블 vs 단일 모델 설명
print("\n🧠 모델 학습 방식:")
print("   • 단일 모델: 빠름, 테스트용")
print("   • 앙상블 (5모델): 논문 방식, 더 안정적 성능")
print("     - 동일 모델을 5번 독립 훈련")
print("     - 예측 시 5개 결과 평균화")
print("     - 확률적 변동성 감소")

# 파일 크기 요약
if os.path.exists('models'):
    print("\n🧠 학습된 모델:")
    model_files = [f for f in os.listdir('models') if f.endswith('.tar')]
    
    # 단일 모델
    single_models = [f for f in model_files if '_run' not in f]
    if single_models:
        print("   📁 단일 모델:")
        for file in sorted(single_models):
            size_mb = os.path.getsize(f'models/{file}') / (1024**2)
            print(f"     ✅ {file} ({size_mb:.1f}MB)")
    
    # 앙상블 모델
    ensemble_models = [f for f in model_files if '_run' in f]
    if ensemble_models:
        print("   📁 앙상블 모델:")
        for file in sorted(ensemble_models):
            size_mb = os.path.getsize(f'models/{file}') / (1024**2)
            print(f"     ✅ {file} ({size_mb:.1f}MB)")

# 논문 성과와 비교
print("\n📚 논문 예상 성과 (비교 참조):")
print("   • Weekly (I5R5): H-L Sharpe = 7.15")
print("   • Monthly (I20R20): H-L Sharpe = 2.16") 
print("   • Quarterly (I60R60): H-L Sharpe = 0.37")

print("\n🔧 사용된 형식:")
print("   • 원본 형식 (.dat + .feather): 논문 저자와 동일")
print("   • 메모리 효율적이고 Colab 친화적")

print("\n✅ 포트폴리오 평가 결과를 확인하여 논문과 비교해보세요!")
print("\n💾 최종 GPU 메모리 상태:")
check_gpu_memory()